In [1]:
import keras
from keras.layers import Input, ReLU,Dense,Dropout,Conv1D,MaxPooling1D,UpSampling1D,BatchNormalization
from keras.models import Model
from keras.optimizers import RMSprop
from keras import backend as K
import numpy as np
from keras.layers import Flatten
import pandas as pd
import random

Using TensorFlow backend.


In [3]:
class_num=4
speed1=1000
speed2=800
#trainx=pd.read_csv(f'../data/CRWU/x_{speed1}_{class_num}_f.csv')
#trainy=pd.read_csv(f'../data/CRWU/y_{speed1}_{class_num}.csv')
#x_test=pd.read_csv(f'../data/CRWU/x_{speed2}_{class_num}_f.csv').values
#y_test=pd.read_csv(f'../data/CRWU/y_{speed2}_{class_num}.csv').values
trainx=pd.read_csv(f'../data/JN/x_{speed1}_2_f.csv').iloc[:4000]
trainy=pd.read_csv(f'../data/JN/y_{speed1}_2.csv').iloc[:4000]
x_test=pd.read_csv(f'../data/JN/x_{speed2}_2_f.csv').iloc[:4000].values
y_test=pd.read_csv(f'../data/JN/y_{speed2}_2.csv').iloc[:4000].values
index =list(trainx.index)
random.Random(0).shuffle(index)
num=int(len(index)/5*5)
x_train,y_train=trainx.iloc[index[:num],:].values,trainy.iloc[index[:num],:].values
#x_test,y_test=trainx.iloc[index[num:],:].values,trainy.iloc[index[num:],:].values
#data_X=np.vstack([x_train,x_test])
def trans(data):
    nm=np.zeros(data.shape)
    for i in range(len(data)):
        nm[i,:]=(data[i,:])/np.max(data[i,:])
    return nm
x_train=trans(x_train)
x_test=trans(x_test)
data_X=np.vstack([x_train,x_test])

In [3]:
x_train1D=x_train.reshape(x_train.shape[0],1024,1).astype('float32')
x_test1D=x_test.reshape(x_test.shape[0],1024,1).astype('float32')
data_X1D=data_X.reshape(len(data_X),1024,1).astype('float32')

In [14]:

K.clear_session()

my_init=keras.initializers.glorot_uniform(seed=2020)
input_image=Input((1024,1))
#编码器 2个简单的卷积，2个最大池化，缩小为原来的1/4了
encoder=Conv1D(16,32,padding='same',activation='relu',kernel_initializer=my_init)(input_image)
encoder=MaxPooling1D(4)(encoder)
#encoder=BatchNormalization()(encoder)
encoder=Conv1D(8,10,padding='same',activation='relu',kernel_initializer=my_init)(encoder)
encoder_out=MaxPooling1D(4)(encoder)
#encoder_out=BatchNormalization()(encoder_out)
x_=Flatten()(encoder_out)
LR=Dense(class_num,activation='softmax',name='LR')(x_)                                                               
# 构建编码模型，可以提取特征图片，展开了就是特征向量
encoder_model = Model(inputs=input_image, outputs=encoder_out)

#解码器，反过来                                                             
decoder=UpSampling1D(4)(encoder_out)                                                                 
decoder=Conv1D(8,10,padding='same',activation='relu',kernel_initializer=my_init)(decoder)                                               
decoder=UpSampling1D(4)(decoder)
decoder=Conv1D(16,32,padding='same',activation='relu',kernel_initializer=my_init)(decoder)
#转成原始图片尺寸
decoder_out=Conv1D(1,10, padding='same',activation='sigmoid',kernel_initializer=my_init)(decoder)
                                                                
autoencoder=Model(input_image,decoder_out)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.fit(data_X1D, data_X1D, epochs=5, batch_size=248, shuffle=True,verbose=1)

In [15]:
encoder=Model(inputs=input_image,outputs=LR)
encoder.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
encoder.fit(x_train1D,y_train,epochs=50,batch_size=248,shuffle=True)
score=encoder.evaluate(x_test1D,y_test)
print(score)

In [ ]:
testcnn_d=encoder.predict(x_test1D)
def save_data(data,name):
    sdata=pd.DataFrame(data,columns=list(range(data.shape[1])))
    sdata.to_csv(f'./prob/{name}.csv',encoding='utf-8',index=None)
save_data(testcnn_d,f'{speed1}_{speed2}_cnn')
encoder.save(f'./model/model_{speed1}_{speed2}_cnn_m.h5')
encoder.save_weights(f'./model/model_{speed1}_{speed2}_cnn_w.h5')
with open('predict_prob.txt','a+') as f:
    f.write(f'cnn_{speed1}_{speed2}'+'----'+str(score)+'\n')

In [16]:
model_json = encoder.to_json()
with open(f'./model/model_cnn_m10.json', 'w') as file:
    file.write(model_json)